In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataclasses import dataclass

import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx

from potentiel_solaire.constants import DEFAULT_CRS, CRS_FOR_BUFFERS
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.extract import extract_sources
from potentiel_solaire.sources.schools_establishments import get_schools_establishments_of_interest
from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools

In [ ]:
code_departement = "093"

In [ ]:
sources = extract_sources()
    
bd_topo_path = extract_bd_topo(code_departement=code_departement)

schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=sources["etablissements"].filepath,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=DEFAULT_CRS
)
nb_schools = schools_establishments.shape[0]

communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(DEFAULT_CRS)
geom_of_interest = communes.sjoin(schools_establishments).dissolve()[["geometry"]]

educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=DEFAULT_CRS
)
nb_educational_zones = educational_zones.shape[0]

buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)
nb_buildings = buildings.shape[0]

In [ ]:
schools_buildings = attach_buildings_to_schools(
    schools_establishments=schools_establishments,
    educational_zones=educational_zones,
    buildings=buildings
)

In [ ]:
@dataclass
class ItemsOfInterest:
    educational_zones: list[str]
    buildings: list[str]
    schools: list[str]

    
def get_facecolor_nb_rattachements(value):
    if value == 0:
        return "yellow"
    elif value == 1:
        return "green"
    else:
        return "red"


def plot_zone_of_interest(
    zone_of_interest: gpd.GeoDataFrame,
    edu_zones: gpd.GeoDataFrame = educational_zones,
    topo_buildings: gpd.GeoDataFrame = buildings,
    schools: gpd.GeoDataFrame = schools_establishments,
    schools_buildings_attached: gpd.GeoDataFrame = schools_buildings,
    buffer_size: int = 5,
):  
    fig, ax = plt.subplots(figsize=(20, 10))
    
    buildings_in_zone = gpd.sjoin(
        topo_buildings, zone_of_interest[["geometry"]], how='inner', predicate='intersects'
    )
    
    buildings_attach_count = schools_buildings_attached.groupby(
        by="cleabs_bat", as_index=False
    ).identifiant_de_l_etablissement.count().rename(
        columns={"identifiant_de_l_etablissement": "nb_rattachements"}
    )
    buildings_in_zone = buildings_in_zone.merge(
        buildings_attach_count,
        left_on="cleabs",
        right_on="cleabs_bat",
        how="left",
    ).copy()
    buildings_in_zone["nb_rattachements"] = buildings_in_zone["nb_rattachements"].fillna(0)
    buildings_in_zone["facecolor"] = buildings_in_zone["nb_rattachements"].apply(get_facecolor_nb_rattachements)
    
    buildings_in_zone.plot(
        ax=ax, edgecolor="black", linewidth=0.5, facecolor=buildings_in_zone["facecolor"], alpha=0.5
    )

    # for x, y, label in zip(
    #         buildings_in_zone.geometry.centroid.x, 
    #         buildings_in_zone.geometry.centroid.y, 
    #         buildings_in_zone['cleabs']
    #     ):
    #     ax.text(x, y, label, fontsize=10, ha='right', color='black')
    
    educational_zones_in_zone = gpd.sjoin(
        edu_zones, zone_of_interest[["geometry"]], how='inner', predicate='intersects'
    )
    educational_zones_in_zone.plot(
        ax=ax, color="none", edgecolor='blue', linewidth=2
    )

    for x, y, label in zip(
            educational_zones_in_zone.geometry.centroid.x, 
            educational_zones_in_zone.geometry.centroid.y, 
            educational_zones_in_zone['identifiants_sources']
        ):
        ax.text(x, y, label, fontsize=10, ha='right', color='blue')
    
    zone_buffered = zone_of_interest[["geometry"]].copy().to_crs(CRS_FOR_BUFFERS)
    zone_buffered.geometry = zone_buffered.geometry.buffer(buffer_size)
    zone_buffered = zone_buffered.to_crs(DEFAULT_CRS)
    
    schools_in_zone = gpd.sjoin(
        schools, zone_buffered, how='inner', predicate='intersects'
    )
    schools_in_zone.plot(
        ax=ax, color="blue", alpha=1
    )

    for x, y, label in zip(
            schools_in_zone.geometry.centroid.x, 
            schools_in_zone.geometry.centroid.y, 
            schools_in_zone['identifiant_de_l_etablissement']
        ):
        ax.text(x, y, label, fontsize=10, ha='right', color='blue')
    
    cx.add_basemap(ax, crs=DEFAULT_CRS, zoom=19, alpha=0.5)
    
    plt.show()
    
    print(f"Duplicated buildings in zone_of_interest: {buildings_in_zone[buildings_in_zone['nb_rattachements'] > 1][['cleabs_bat', 'nb_rattachements']]}")
    
    return ItemsOfInterest(
        educational_zones=list(educational_zones_in_zone["cleabs"].unique()), 
        buildings=list(buildings_in_zone["cleabs"].unique()),
        schools=list(schools_in_zone["identifiant_de_l_etablissement"].unique()),
    )

### 1. Overlap entre plusieurs zones d education

In [ ]:
zone_of_interest_1 = educational_zones[educational_zones["cleabs"].isin(["SURFACTI0000000002555603"])].dissolve()
items_1 = plot_zone_of_interest(
    zone_of_interest=zone_of_interest_1
)

### 2. Batiment overlap plusieurs zones d'educations

In [ ]:
zone_of_interest_2 = educational_zones[educational_zones["cleabs"].isin(["SURFACTI0000000002555878", "SURFACTI0000000002555879", "SURFACTI0000000244244919"])].dissolve()
items_2 = plot_zone_of_interest(zone_of_interest=zone_of_interest_2)

### 3. Batiment overlap à peine la zone d'education

In [ ]:
zone_of_interest_3 = educational_zones[educational_zones["cleabs"].isin(["SURFACTI0000000002555651"])].dissolve()
items_3 = plot_zone_of_interest(zone_of_interest=zone_of_interest_3)

### 4. Etablissements au meme endroit et overlap des zones d'éducation associees

In [ ]:
zone_of_interest_4 = educational_zones[educational_zones["cleabs"].isin(["SURFACTI0000000002555609", "SURFACTI0000002215707916", "SURFACTI0000000002555601", "SURFACTI0000000002555600", "SURFACTI0000000002555602"])].dissolve()
items_4 = plot_zone_of_interest(zone_of_interest=zone_of_interest_4)

### 5. Cas d echec le plus frequent de la premiere methode de rattachement

In [ ]:
zone_of_interest_5 = educational_zones[educational_zones["cleabs"].isin(["SURFACTI0000000351259092"])].dissolve()
items_5 = plot_zone_of_interest(zone_of_interest=zone_of_interest_5)

### 6. Une zone est partagée entre plusieurs établissements

In [ ]:
zone_of_interest_6 = educational_zones[educational_zones["cleabs"].isin(["SURFACTI0000000002555427"])].dissolve()
items_6 = plot_zone_of_interest(zone_of_interest=zone_of_interest_6)

In [ ]:
import json 

def save_cases_to_json(
    items: ItemsOfInterest, 
    filename: str
):
    schools = schools_establishments.copy()
    zones = educational_zones.copy()
    builds = buildings.copy()

    # convert geometries to WKT format
    schools["geometry"] = schools["geometry"].apply(lambda x: x.wkt)  
    zones["geometry"] = zones["geometry"].apply(lambda x: x.wkt)
    builds["geometry"] = builds["geometry"].apply(lambda x: x.wkt)
    
    items_schools = schools[schools["identifiant_de_l_etablissement"].isin(items.schools)][[
        "identifiant_de_l_etablissement",
        "geometry",
    ]].to_dict('list') 

    items_zones = zones[zones["cleabs"].isin(items.educational_zones)][[
        "cleabs",
        "identifiants_sources",
        "geometry",
    ]].to_dict('list')

    items_buildings = builds[builds["cleabs"].isin(items.buildings)][[
        "cleabs",
        "geometry",
    ]].to_dict('list')

    items_dict = {
        "schools_establishments": items_schools,
        "educational_zones": items_zones,
        "buildings": items_buildings,
    }

    with open(filename, 'w') as f:
        json.dump(items_dict, f, indent=4, ensure_ascii=False)


In [ ]:
for i, item in enumerate([items_1, items_2, items_3, items_4, items_5, items_6]):
    save_cases_to_json(
        items=item,
        filename=f"cases_{i+1}.json"
    )